In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
import pickle

Using TensorFlow backend.


In [2]:
from notebook.services.config import ConfigManager
ConfigManager().update('notebook', {'ExecuteTime': {
   	'display_absolute_timestamps': False,
    'relative_timing_update_period': 5,
    'template': {
    	'executed': 'started ${start_time}, finished in ${duration}',
    }
}})

{'Cell': {'cm_config': {'lineNumbers': True}},
 'ExecuteTime': {'display_absolute_timestamps': False,
  'relative_timing_update_period': 5,
  'template': {'executed': 'started ${start_time}, finished in ${duration}'}}}

## Finding composition with Enrichment and Burnup
Testing time for various methods

In [3]:
sample_num = 100
enr = np.random.uniform(low=3, high=5, size=(sample_num,))
br = np.random.uniform(low=30000, high=50000, size=(sample_num,))
enr_br = np.column_stack((enr,br))
print(np.shape(enr_br))
enr_br

(100, 2)


array([[  4.47014036e+00,   4.72988918e+04],
       [  3.90171511e+00,   3.02333293e+04],
       [  4.53933611e+00,   3.04706673e+04],
       [  3.40513114e+00,   3.74094223e+04],
       [  3.46199764e+00,   4.51302544e+04],
       [  4.95168261e+00,   3.76590567e+04],
       [  4.76320862e+00,   4.44573653e+04],
       [  4.70066734e+00,   4.97127823e+04],
       [  4.97005132e+00,   4.02134762e+04],
       [  4.18022070e+00,   4.91111367e+04],
       [  4.88634845e+00,   4.63515599e+04],
       [  4.13912453e+00,   3.51911490e+04],
       [  3.44267586e+00,   4.81089460e+04],
       [  4.81314160e+00,   4.80878302e+04],
       [  3.93632102e+00,   3.66123388e+04],
       [  4.22085774e+00,   3.59972764e+04],
       [  3.96057819e+00,   4.52436912e+04],
       [  4.93500065e+00,   4.66702123e+04],
       [  3.12683047e+00,   4.30463320e+04],
       [  4.80326854e+00,   3.19167503e+04],
       [  4.94195482e+00,   4.16819367e+04],
       [  3.25912855e+00,   3.37228635e+04],
       [  

## Pickle and model

In [4]:
# load pickled dictionary with
# key: isotope
# value: trained model
model_dict = pickle.load(open('./script/ann_model.pkl',"rb"))

In [5]:
# get time for rom depletion for one assembly
# given enrichment and burnup value,
# predict UNF composition

def predict(enr_bu):
    model = model_dict['model']
    iso_list = model_dict['iso_list']
    x = model_dict['xscaler'].transform(enr_bu)
    y = model_dict['yscaler'].inverse_transform(model.predict(x))
    comp_dict = {}
    for indx, i in enumerate(y):
        comp_dict[indx] = {}
        for indx2, iso in enumerate(iso_list):
            comp_dict[indx][iso] = i[indx2]
    return comp_dict


%time predict(enr_br)


CPU times: user 82.1 ms, sys: 14.9 ms, total: 96.9 ms
Wall time: 120 ms


{0: {'zr-95': 0.00014333532,
  'zr-93': 0.10864968,
  'ra-226': 7.1895051e-12,
  'ru-103': 8.3539535e-06,
  'ru-106': 0.010722049,
  'sb-124': 1.2478847e-07,
  'sb-125': 0.0011916951,
  'pu-238': 0.037054934,
  'pu-239': 0.74270165,
  'np-237': 0.073723748,
  'nb-95': 0.00016908877,
  'pa-231': 7.1749724e-08,
  'pd-107': 0.034079827,
  'pm-147': 0.016188357,
  'pu-240': 0.28970525,
  'pu-241': 0.1947196,
  'pu-244': 8.365012e-06,
  'pu-242': 0.079833493,
  'i-129': 0.022659885,
  'h-3': 7.4574714e-06,
  'kr-85': 0.0035193088,
  'nb-94': 2.329223e-07,
  'cm-248': 1.600221e-07,
  'cs-134': 0.015434409,
  'cs-135': 0.065108627,
  'cm-245': 0.00093407027,
  'cm-244': 0.010883128,
  'cm-246': 9.8390265e-05,
  'cm-247': 1.7914723e-06,
  'eu-152': 2.0010614e-06,
  'cs-137': 0.16595374,
  'eu-154': 0.0041152816,
  'eu-155': 0.001106569,
  'am-241': 0.017786536,
  'am-242m': 0.00013690465,
  'am-243': 0.023854904,
  'cf-251': 4.8955129e-10,
  'cf-249': 1.6317979e-09,
  'ag-108m': 3.4855777e-08,

## Looking up from db

In [6]:

df = pd.read_csv('./curated.csv', index_col=0)

In [7]:
# get time for `look-up' method for one assembly
# find closest assembly to this burnup and enrichment
def deplete_lookup(df, enr_br):
    depleted_composition_lookup = {}
    for indx, enr_br_set in enumerate(enr_br):
        df['err'] = abs(df['bu'] - enr_br[0][1]) + abs(df['init_enr'] - enr_br[0][0]) * 10000
        minimum_err = df['err'].argmin()
        df = df.drop(columns='err')
        average_assem = df.loc[minimum_err]
        depleted_composition_lookup[indx] = average_assem.iloc[5:].to_dict()
    return depleted_composition_lookup

%time deplete_lookup(df, enr_br)

/home/teddy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys


CPU times: user 7.51 s, sys: 14.4 s, total: 21.9 s
Wall time: 21.8 s


{0: {'zr-95': 0.0001811262814375708,
  'zr-93': 0.10954227159263293,
  'ra-226': 3.8908634867342515e-12,
  'ru-103': 1.0252945612401473e-05,
  'ru-106': 0.012372715926329086,
  'sb-124': 1.4572032857261509e-07,
  'sb-125': 0.0013305827617411373,
  'pu-238': 0.036110989433741206,
  'pu-239': 0.82207825676402313,
  'np-237': 0.078610593800137268,
  'nb-95': 0.00021363518210074886,
  'pa-231': 5.6457420206603799e-08,
  'pd-107': 0.032759972808386564,
  'pm-147': 0.018042640996384651,
  'pu-240': 0.28594263531325004,
  'pu-241': 0.2096873751349744,
  'pu-244': 8.2247559530835371e-06,
  'pu-242': 0.073481234726575639,
  'i-129': 0.022622565541843168,
  'h-3': 7.4437146716678236e-06,
  'kr-85': 0.0039611022221056457,
  'nb-94': 2.5792036616873661e-07,
  'cm-248': 1.0999237804211636e-07,
  'cs-134': 0.016918801120889015,
  'cs-135': 0.061770598084346449,
  'cm-245': 0.00087442368643086011,
  'cm-244': 0.0098490061071839184,
  'cm-246': 7.6052210520793706e-05,
  'cm-247': 1.3387297756910471e-0